# Classification pipeline for a single glacier

Full glacier-snow-cover-mapping classification pipeline for Sentinel-2 TOA, Sentinel-2 SR, and Landsat 8/9 images. 

## Requirements:
1. Google Earth Engine (GEE) account: used to query imagery and the DEM (if no DEM is provided). Sign up for a free account [here](https://earthengine.google.com/new_signup/). 

2. Google Drive folder: Create a folder where output snow cover statistics will be saved. Enter the name of this folder as the `out_folder` variable below. If you don't create the folder ahead of time, duplicates of the same folder will be created for each output file!

## Notes on GEE job submissions

GEE enforces [user quotas](https://developers.google.com/earth-engine/guides/usage) on memory usage (default = 10 MB per query) and the number of concurrent requests (default = 40). The workflow below mitigates the risk of exceeding these limits by splitting tasks into smaller date ranges.While GEE handles queued tasks efficiently, it's our job to manage the size of the individual jobs (i.e., image collections). To reduce task size, the date range is divided before running the workflow and submitting export tasks based on the size of the defined area of interest (AOI):

- AOI < 500 km$^2$: split by month
- 700 km$^2$ <= AOI < 1100 km$^2$: split by week
- AOI >= 1100 km$^2$: split by day

Especially for the largest glaciers, this means that some exports may be empty, depending on image availability. To compile all results and remove empty CSVs from your Google Drive folder, see the `post_processing.ipynb` notebook in this repo. 

## Define image search settings and paths

In [ ]:
import os
import ee
import geemap
import sys
import numpy as np

# -----Define Google Drive folder for outputs
# Note: Make sure this folder already exists and is the only folder in your "My Drive" with that name. 
out_folder = 'glacier_snow_cover_exports'

# -----Import pipeline utilities
# Assumes pipeline_utils.py is in the same folder as this notebook
script_path = os.getcwd()
sys.path.append(script_path)
import pipeline_utils as utils

# -----Define image search settings
# Date and month ranges (inclusive)
date_start = '2013-06-01'
date_end = '2024-10-31'
month_start = 6
month_end = 10
# Minimum fill portion of the AOI (0–100), used to remove images after mosaicking by day. 
min_aoi_coverage = 70
# Whether to mask clouds using the respective cloud mask via the geedim package
mask_clouds = True

## Authenticate and/or Initialize Google Earth Engine (GEE)

Replace the project ID with your GEE project. Default = `ee-{GEE-username}`

In [ ]:
# project_id = "snow-cover-mapping-463217"
project_id = "ee-raineyaberle"

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()
    ee.Initialize(project=project_id)

## Select the Area of Interest (AOI) from the GLIMS dataset

This cell will plot the GLIMS dataset on a map. To find a glacier, click on the wrench in the upper right toolbox of the map, and use the "Inspector" to click on a polygon and view the its properties. Right click on the "glac_id" property to highlight and then copy. Replace the `glac_id` variable below with your selected site. 

In [ ]:
# Load the GLIMS dataset, add to the map
glims = ee.FeatureCollection('GLIMS/20230607')

# Select your study site from the GLIMS dataset
# glac_id = 'G211100E60420N'
# aoi = glims.filter(ee.Filter.eq('glac_id', glac_id))

# => Test with the largest AK glacier (Malaspina)
glims_ak = (glims.filter(ee.Filter.eq('gtng_o1reg', 1))) # filter to AK
glims_ak_largest = glims_ak.filter(ee.Filter.eq('area', glims_ak.aggregate_max('area'))) # Get the largest AK glacier
aoi = glims_ak_largest
# use GLIMS ID for output file names
if type(aoi)==ee.featurecollection.FeatureCollection:
    glac_id = ee.FeatureCollection(aoi).aggregate_array('glac_id').get(0).getInfo()
else:
    glac_id = aoi.get('glac_id').getInfo() 
print("Glacier ID used for output file names:", glac_id)
                                 
# Grab the geometry
aoi = aoi.geometry()
aoi_area = aoi.area().getInfo() # save area [m^2] for splitting date ranges later
print(f"Glacier area = {int(aoi_area/1e6)} km2")

# Create a Map
Map = geemap.Map()
Map.addLayer(glims, {'color': 'blue', 'opacity':  0.5}, 'GLIMS/20230607')
Map.addLayer(aoi, {'color': 'orange', 'opacity': 0.8}, 'AOI')
Map.centerObject(aoi)
Map

## Load the Digital Elevation Model (DEM)

Default: use the ArcticDEM Mosaic where there is > 90 % coverage. Otherwise, use the NASADEM. For sites that use the ArcticDEM Mosaic, elevations are reprojected to the EGM96 geoid to match the vertical datum of NASADEM. 

In [ ]:
# Query GEE for DEM
dem = utils.query_gee_for_dem(aoi)

# Add DEM to map
# grab min and max elevations for color limits
minMax = dem.reduceRegion(reducer=ee.Reducer.minMax(),
                          geometry=aoi, 
                          scale=30,
                          maxPixels=1e9,
                          bestEffort=True)
elev_min = minMax.get('elevation_min')
elev_max = minMax.get('elevation_max')
print(f'Elevation range = {int(elev_min.getInfo())} to {int(elev_max.getInfo())} m')
# colors based on the "terrain" palette from matplotlib
palette = ['#333399', '#0d7fe5', '#00be90','#55dd77','#c6f48e','#e3db8a','#aa926b','#8e6e67','#c6b6b3','#ffffff']
Map.addLayer(dem, {'palette': palette, 'min': elev_min, 'max': elev_max}, 'DEM')

## Run the classification pipeline for each dataset

### Sentinel-2 Top of Atmosphere (TOA): 2016 onwards

In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
dataset = "Sentinel-2_TOA"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

### Sentinel-2 Surface Reflectance (SR): 2019 onwards

In [ ]:
dataset = "Sentinel-2_SR"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

### Landsat 8/9 SR: 2013 onwards

In [ ]:
dataset = "Landsat"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

_Fin!_